In [49]:
from prompt.template import PromptTemplate
from prompt.store import TemplateStore

prompt_store = TemplateStore()

prompt_store.add_template(
        template_id="CI_SYSTEM_PROMPT",
        template=PromptTemplate(
            template=""" You are a helpful AI assistant.

You have access to a {display_name} code interpreter, which supports you in your tasks.
The code is executed in an interactive shell, imports and variables are preserved between calls.
The environment has internet and file system access.
The current working directory is shared with the user, so files can be exchanged.
There are many libraries pre-installed, including numpy, pandas, matplotlib, and scikit-learn.
You cannot show rich outputs like plots or images, but you can store them in the working directory and point the user to them.
If the code runs too long, there will be a timeout.

To access the interpreter, use the following format:
```{tag_name}
<your code>
```
If you want to call {display_name} and still say something, do only output text above the code block, NOT below.
Only provide at most one code block per message.
The code will be executed automatically and the result will be sent back to you
""",
            params=["display_name", "tag_name"]
        )
)


prompt_store.add_template(
        template_id="CI_AGENT_REPLY",
        template=PromptTemplate(
            template=""" Thank you for the explanation. I understand I have access to a {display_name} interpreter and can execute code by placing it between ```{tag_name} tags. I will use this to assist you with any {display_name} programming tasks. Just let me know what you need help with!
""",
            params=["display_name", "tag_name"]
        )
)


prompt_store.add_template(
        template_id="CI_SCRIPT_ERROR",
        template=PromptTemplate(
            template=" The script failed with below error, correct the script for me. Make sure to include full script between ```{tag_name} tags:\n<error>\n{error_message}\n</error>",
            params=["tag_name", "error_message"]
        )
)


prompt_store.add_template(
        template_id="CI_SCRIPT_SUCCESS",
        template=PromptTemplate(
            template=" The script executed successfully with below output, Answer with only Yes or No if this is the expected output:\n<output>\n{script_output}\n</output>",
            params=["script_output"]
        )
)

In [59]:
ROLES = ["Human", "Assistant"]

LANGUAGES = {
    "Python": {
        "display_name": "Python",
        "kernel_name": "python3",
        "tag_name": "python"
    },
    "R": {
        "display_name": "R",
        "kernel_name": "ir",
        "tag_name": "R"
    },
    "Java": {
        "display_name": "Java",
        "kernel_name": "java",
        "tag_name": "java"
    },
}

In [26]:
!docker context ls

NAME                             DESCRIPTION                               DOCKER ENDPOINT                                              ERROR
default                          Current DOCKER_HOST based configuration   unix:///var/run/docker.sock                                  
m5.large_i-02880917500cf72a3 *                                             tcp://ip-172-31-79-18.ap-southeast-2.compute.internal:1111   


In [67]:
import requests
import json

docker_daemon = "ip-172-31-79-18.ap-southeast-2.compute.internal"

api_layer_addr = f"http://{docker_daemon}:8001"
api_layer_url = api_layer_addr + "/invoke"
model_family = "bedrock"
model_name = "anthropic.claude-v2"

code_executor_addr = f"http://{docker_daemon}:8080"
code_executor_url = code_executor_addr + "/execute_code"

code_block_symbol = "```"


def send_req_to_agent(text):
    data = {
        "body": {
            "prompt": text
        }, 
        "model_family": model_family, 
        "model_name": model_name
    }
    ret = requests.post(
        url=api_layer_url, 
        data=json.dumps(data)
    )
    return json.loads(ret.text)["generated_text"]

def send_script_to_exc(script, kernel_name):
    data = {
        "code": script, 
        "kernel_name": kernel_name
    }
    ret = requests.post(
        url=code_executor_url, 
        data=json.dumps(data)
    )
    return json.loads(ret.text)

def extract_script(text, tag_name):
    try:
        if code_block_symbol + tag_name in text:
            script = text.split(code_block_symbol + tag_name)[1].split(code_block_symbol)[0]
            return script
    except:
        return None
    return None
    

class Conversation:
    def __init__(
        self, 
        roles, 
        prompt, 
        agent, 
        executor,
        script_extractor,
        language
    ):
        self.roles = roles
        self.system_prompt = prompt
        self.history = "\n"
        self.append_chat(prompt, 0)
        self.agent = agent
        self.executor = executor
        self.script_extractor = script_extractor
        self.language = language
        self.last_agent_message = ""
        
    def append_chat(self, text, role=0):
        self.history += "\n" + self.roles[role] + ":" + text
        
    def send_to_agent(self):
        self.append_chat("", 1)
        res = self.agent(self.history)
        self.history += res
        self.last_agent_message = res
    
    def send_to_agent_and_exec_script(self):
        self.send_to_agent()
        script = self.script_extractor(
            self.last_agent_message,
            LANGUAGES[self.language]["tag_name"]
        )
        if script:
            res = self.executor(script, LANGUAGES[self.language]["kernel_name"])
            return res
        else:
            return None
        

In [68]:
test_prompts = [
    "I want a script to print 'Hello World!'",
    # "I want a script that compute first 10 odd numbers and print them.",
    # "I want a script that compute factorial of 20 and determines the time it took to calculate the result.",
    # "I want to make tetris game"
]

language = "Java"

params = {
    "display_name": LANGUAGES[language]["display_name"],
    "tag_name": LANGUAGES[language]["tag_name"],
    "error_message": "",
    "script_output": ""
}

for test in test_prompts:
    conv = Conversation(
        ROLES, 
        prompt_store.get_prompt_from_template(
            "CI_SYSTEM_PROMPT",
            params
        ),
        send_req_to_agent,
        send_script_to_exc,
        extract_script,
        language
    )
    conv.append_chat(
        prompt_store.get_prompt_from_template(
            "CI_AGENT_REPLY",
            params
        ),
        1
    )
    conv.append_chat(test)
    res = conv.send_to_agent_and_exec_script()
    max_iterations = 1
    i = 0
    while i < max_iterations and res["error"]:
        i += 1
        params["error_message"] = res["output"]
        conv.append_chat(
            prompt_store.get_prompt_from_template(
                "CI_SCRIPT_ERROR",
                params
            )
        )
        conv.send_to_agent_and_exec_script()   
    if not res["error"]:
        params["script_output"] = res["output"]
        conv.append_chat(
            prompt_store.get_prompt_from_template(
                "CI_SCRIPT_SUCCESS",
                params
            )
        )
        conv.send_to_agent()
    print(conv.history)

{"code": "\npublic class Main {\n  public static void main(String[] args) {\n    System.out.println(\"Hello World!\"); \n  }\n}\n", "kernel_name": "java"}


Human: You are a helpful AI assistant.

You have access to a Java code interpreter, which supports you in your tasks.
The code is executed in an interactive shell, imports and variables are preserved between calls.
The environment has internet and file system access.
The current working directory is shared with the user, so files can be exchanged.
There are many libraries pre-installed, including numpy, pandas, matplotlib, and scikit-learn.
You cannot show rich outputs like plots or images, but you can store them in the working directory and point the user to them.
If the code runs too long, there will be a timeout.

To access the interpreter, use the following format:
```java
<your code>
```
If you want to call Java and still say something, do only output text above the code block, NOT below.
Only provide at most one code block pe

In [54]:
conv.history

'\n\nHuman: You are a helpful AI assistant.\n\nYou have access to a R code interpreter, which supports you in your tasks.\nThe code is executed in an interactive shell, imports and variables are preserved between calls.\nThe environment has internet and file system access.\nThe current working directory is shared with the user, so files can be exchanged.\nThere are many libraries pre-installed, including numpy, pandas, matplotlib, and scikit-learn.\nYou cannot show rich outputs like plots or images, but you can store them in the working directory and point the user to them.\nIf the code runs too long, there will be a timeout.\n\nTo access the interpreter, use the following format:\n```R\n<your code>\n```\nIf you want to call R and still say something, do only output text above the code block, NOT below.\nOnly provide at most one code block per message.\nThe code will be executed automatically and the result will be sent back to you\n\nAssistant: Thank you for the explanation. I underst